In [ ]:
# # Update pip
# sudo apt-get update
# sudo apt-get install -y python3-pip
# sudo -H pip3 install --upgrade pip

# # Install packages
# pip3 install openai
# pip3 install langchain==0.0.100
# pip3 install wikipedia

In [2]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [3]:
llm = OpenAI(temperature=0)

In [8]:
import os
import openai
openai.api_type = "azure"
openai.api_base = ""
openai.api_version = "2022-12-01"
#openai.api_key = os.getenv("66bcffe0ed6c440dba76940d04a7bb03")
openai.api_key = ""

In [9]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.llms import AzureOpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

# build tools
docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# build LLM
llm = AzureOpenAI(
  deployment_name="text-davinci-003",
  model_name="text-davinci-003",
  temperature=0,
)

# initialize ReAct agent
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)

# perform question-answering
question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search David Chanoff and the U.S. Navy admiral, find the ambassador to the United Kingdom, then find the President they served under.
Action: Search[David Chanoff]
Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought: David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom. I need to search U.S. Navy admiral and find the ambassador to the United Kingdom.
Action: Search[U.S. Navy admiral]
Observation:

'Bill Clinton'

In [10]:
# output prompt's template
print(react.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St